In [2]:
import matplotlib.pyplot as plt
import numpy as np
from KDEpy import NaiveKDE
import scipy.integrate as integrate
from function_library_3 import trim
from tqdm import tqdm

#================================================================================
# SETUP

#change default font size for graphs
plt.rcParams.update({'font.size':20})

# create the needed directories and clear old output
import os,glob
try:
    os.mkdir('3_Pressure_Distributions')
except:
    for filename in glob.glob('3_Pressure_Distributions/*'):
        os.remove(filename)
try:
    os.mkdir('3_Temperature_Distributions')
except:
    for filename in glob.glob('3_Temperarture_Distributions/*'):
        os.remove(filename)


# suppress warnings
import warnings
warnings.filterwarnings('ignore')


#================================================================================
# RETRIEVE BASIN SIZE DATA

# open the basin input file
input_file = open('Basins.txt','r')

# create a hashmap from basin name -> diameter
basin_diameter = {} 
line = ' '
while line[0:3] != 'END':
    line = input_file.readline()
    if line[0:2] == 'LB': # data is retrieved from lines starting with LB: (Lunar Basin)
        name, size, N, E = trim(line.split(':'))
        basin_diameter[name] = size
        
input_file.close()
    
#================================================================================
# DEFINE HAVERSINE FORMULA
    
def haversine(lat1, lon1, lat2, lon2, Radius): # haversine formula
    dLat = np.radians(lat2 - lat1) # change in latitude
    dLon = np.radians(lon2 - lon1) # change in longitude
    lat1 = np.radians(lat1) # latitude 1
    lat2 = np.radians(lat2) # latitude 2
    a = np.sin(dLat/2)**2 + np.cos(lat1)*np.cos(lat2)*np.sin(dLon/2)**2 # intermediate step
    c = 2*np.arcsin(np.sqrt(a)) # interior angle between two coordinates [rad]
    return Radius * c # returns distance between two coordinates

#================================================================================
# GLOBAL PARAMETERS

p_Range = [.1,310] # plotted pressure range
T_Range = [200,14000] # plotted temperature range
    
basin_list = [] # generate list of all basins considered
for filename in os.listdir('1_Basin_Data'):
    if filename.endswith(".dat"):
        basin_list.append(filename)

#================================================================================
# DEFINE CROSS-SECTION FUNCTION

def cross_section(loc, param):  # take a slice of the pressure and temperature 2D KDEs  
    # basin name    2D KDE values    Stacked Height values   Loc of basin, Radius planet, Radius basin 
    basin,         x, yp,zp, yT,zT,   height_x, height_y,                   coords, R_p, R_c           = param
    #             loc  pres   Temp     loc       height                      N,E
    
    #---------------------------------------
    # data is interpolated linearly between the two surounding columns closest the to landing location
    
    lower = min(x) ; upper = max(x) # find the lower and upper bounds of the landing location data
    
    # find the below and above cross-section
    for i in range(len(x)):
        if loc < x[i] <= upper:
            upper = x[i]
        if lower < x[i] <= loc:
            lower = x[i]
    # if the location is out of bounds, return null values       
    if lower == upper or max(height_x) < loc:
        return 0,[],[],[],[]
    
    #---------------------------------------
    # interpolate stacked height curve 
    i=0 ; height_accum = 0
    while i+1 < len(height_x):
        if height_x[i] <= loc <= height_x[i+1]: # locate the two points that need to be interpolated
            xl=height_x[i] ; xh=height_x[i+1] ; c=loc # x values of data below and above the loc
            yl=height_y[i] ; yh=height_y[i+1] # y values of data below and above
            height_accum = yl+(yh-yl)/(xh-xl)*(c-xl) # linearly interpolate
            break
        i+=1  
    
    #---------------------------------------
    # initialize lists of the cross-sections below and above the landing location
    pres0,pres1 = [],[] ; Temp0,Temp1 = [],[] # pressures, probabilty density ; pressures, probabilty density 
    pval0,pval1 = [],[] ; Tval0,Tval1 = [],[] # temperature, probabilty density ; temperature, probabilty density 

    # create the lists of the cross-sections above and below the landing location 
    for i in range(len(x)):
        if x[i] == lower: # below cross-sections
            pres0.append(yp[i])
            pval0.append(zp[i])
            Temp0.append(yT[i])
            Tval0.append(zT[i])
        if x[i] == upper: # above cross-sections
            pres1.append(yp[i])
            pval1.append(zp[i])
            Temp1.append(yT[i])
            Tval1.append(zT[i])
    
    # intialize lists of the cross-section at the landing location
    pres,pval=[],[] # pressures, probabilty density
    Temp,Tval=[],[] # temperature, probabilty density
    
    # linearly interpolate the below and above cross-section curves to get the desired cross-section      
    for i in range(len(pres0)):
        pres.append(pres0[i]) # append the pressures 
        Temp.append(Temp0[i]) # append the temperatures 
        
        xl=lower ; xh=upper ; c=loc # locate the two cross-sections that need to be interpolated
        
        # interpolate the pressure cross-sections
        yl=pval0[i] ; yh=pval1[i]
        value = yl+(yh-yl)/(xh-xl)*(c-xl) # linearly interpolate
        pval.append(value)
        
        # interpolate the temperature cross-sections
        yl=Tval0[i] ; yh=Tval1[i]
        value = yl+(yh-yl)/(xh-xl)*(c-xl) # linearly interpolate
        Tval.append(value)
        
    #---------------------------------------
    # returns a stacked height value, pressure curve, temperature curve
    return height_accum, np.array(pres), np.array(pval), np.array(Temp), np.array(Tval)

#================================================================================
# DEFINE MAIN FUNCTION

# Open the user input file with the information from each basin
input_file = open('Sample_Sites.txt','r')
samples = []
line = 1,' '
while line[0:3] != 'END':
    line = input_file.readline()
    if line[0:1] == 'S':
        samples.append(trim(line.split(':'))) 

def distribution(specific_sample_site): # create pressure distributions at the sample site
    
    title_loc,N_sample,E_sample = specific_sample_site
    
    print('============================================================================')
    print(title_loc)
    print('============================================================================')
    
    #---------------------------------------
    # begin combining the pressure/temperature distributions
    
    # initialize lists 
    names = [] ; radii = [] ; distances = [] # names of each basin, diameter of basin, and distance from sample site to center of basin
    heights, px, pv, Tx, Tv = [],[],[],[],[] 
    # respective stacked heights and pressure/temperature distributions from each basin
    
    # cycle through each basin for their contribution
    for each in tqdm(basin_list):
        # extract the data from the KDE data
        fileKDE  = open('2_KDE_Data/{}'.format(each),'r')
        full_data = eval(fileKDE.readline())
        fileKDE.close()
        names.append(full_data[0]) # names of each basin
        N_basin = full_data[8][0] # latitude of basin
        E_basin = full_data[8][1] # longitude of basin
        planet_radius = full_data[9] # radius of planet
        distances.append(haversine(N_basin,E_basin,N_sample,E_sample,planet_radius)) # distance from sample site to center of basin 
        radii.append(full_data[10]) # radius of each basin
        H_0,p_0,pv_0,T_0,Tv_0 = cross_section(distances[-1],full_data) # respective stacked heights and pressure/temperature distributions from each basin
        # add to each list
        heights.append(H_0)
        px.append(p_0) ; pv.append(pv_0)
        Tx.append(T_0) ; Tv.append(Tv_0)
        
    #---------------------------------------
    # synchronize the patterns and colors
    
    import random
    random.seed(7)
    lines = ["--","-.",":"]
    pattern_color = {}
    
    for each in names:
        color="#"+''.join([random.choice('0123456789ABCDEF') for i in range(6)])
        line_type = lines[random.randint(0,2)]
        pattern_color[each] = [line_type,color]
        
    #---------------------------------------
    # sort from nearest to farthest basin (sample to center)
    distances, radii, heights, names , px, pv, Tx, Tv = zip(*sorted(zip(distances, radii, heights, names, px, pv, Tx, Tv)))
    
    # all the px and Tx are identical... so we can collapse that array:
    px = px[0]
    Tx = Tx[0]
    
    # sort the stacked-heights so that they can be ranked
    sorted_heights = []
    for each in heights:
        sorted_heights.append(each)
    sorted_heights.sort() ; sorted_heights.reverse()
    
    # rank the stacked-heights
    i = 0 ; rankings = {}
    for each in sorted_heights:
        if each in rankings.keys():
            rankings[each] = i
        else:
            i += 1
            rankings[each] = i        

    print('Stacked Height at {}'.format(title_loc))
    print('{:0.05f} km of ejecta from all basins'.format(sum(heights)))
    print('----------------------------------------------------------------------------')
    for i in range(len(names)):
        if heights[i] < 1e-9 : # skip basin if it has no contributions
            continue
            
        sample_to_center = '| {:04d} km away '.format(int(distances[i]))
        
        sample_to_edge = '| {:04d} km from edge '.format(int(distances[i] - radii[i]))
        if radii[i] > distances[i]: # determine if the sample site is from within the basin
            sample_to_edge = '| {:04d} km within edge '.format(int(radii[i] - distances[i]))
        
        # print the details from each basin
        rank = rankings[heights[i]]
        print('{:0.05f} km (#{}) | from {:12s}  '.format(heights[i],rank,names[i]) + sample_to_center + sample_to_edge)

    print('============================================================================')
    
    # initialize lists
    p_AbunEach, p_AbunTotal = [], 0
    T_AbunEach, T_AbunTotal = [], 0
    for i in range(len(names)): # add up contributions from each basin
        if heights[i] < 1e-9 : # skip basin if it has no contributions
            p_AbunEach.append([]) # add in dummy placeholders for empty non-contributing basins
            T_AbunEach.append([]) # add in dummy placeholders for empty non-contributing basins
            continue
            
        p_norm = integrate.trapz(pv[i], x=px)
        T_norm = integrate.trapz(Tv[i], x=Tx)
        p_AbunEach.append(heights[i]/p_norm*np.array(pv[i])) # normalize by the stacked height / area under the pressure curve
        T_AbunEach.append(heights[i]/T_norm*np.array(Tv[i])) # normalize by the stacked height / area under the temperature curve 
        p_AbunTotal += p_AbunEach[-1] # combined pressure distribution curve
        T_AbunTotal += T_AbunEach[-1] # combined temperature distribution curve
        
    AreaP = integrate.trapz(p_AbunTotal,x=px) # area under the total pressure curve
    AreaT = integrate.trapz(T_AbunTotal,x=Tx) # area under the total temperature curve
    
    
    #---------------------------------------
    # plot the pressure distributions
    
    fig = plt.figure(figsize=(16,16),facecolor='white')
    plt.suptitle('Pressure Distribution at {}'.format(title_loc), fontsize=32)
    ax = fig.add_subplot(111) 
    
    ax.set_title('Pressure PDF', fontsize=24)
    ax.set_xlabel('Pressure [GPa]', fontsize=24)
    ax.set_ylabel('Probability Density (normed to Combined)', fontsize=24)
    
    ax.plot(px, p_AbunTotal/AreaP, c='black', label='(Combined)', alpha=.5) # plot combined distribution
        
    for i in range(len(names)): # plot each basin's pressure distribution
        if heights[i] < 1e-9 : # skip basin if it has no contributions
            continue
        lsc = pattern_color[names[i]][0]
        col = pattern_color[names[i]][1]
        ax.plot(px, p_AbunEach[i]/AreaP, ls=lsc, c=col, label=names[i])
        
    ax.set_xlim(0,60)
    
    #ax.set_yscale('log')
    
    ax.grid(True, ls='--', zorder=-15); ax.legend(fontsize=24,title='In Order of Proximity');
    
    fig.savefig('3_Pressure_Distributions/Pressure Distribution at {}.png'.format(title_loc))
    plt.close(fig)

    #---------------------------------------
    # plot the temperature distributions
    
    fig = plt.figure(figsize=(16,16),facecolor='white')
    plt.suptitle('Temperature Distribution at {}'.format(title_loc), fontsize=32)
    ax = fig.add_subplot(111) 
    
    ax.set_title('Temperature PDF', fontsize=24)
    ax.set_xlabel('Temperature [K]', fontsize=24)
    ax.set_ylabel('Probability Density (normed to Combined)', fontsize=24)
    
    ax.plot(Tx, T_AbunTotal/AreaT, c='black', label='(Combined)', alpha=.5) # plot combined distribution
    
    for i in range(len(names)): # plot each basin's temperature distribution
        if heights[i] < 1e-9 : # skip basin if it has no contributions
            continue
        lsc = pattern_color[names[i]][0]
        col = pattern_color[names[i]][1]
        ax.plot(Tx, T_AbunEach[i]/AreaT, ls=lsc, c=col, label=names[i])
    
    ax.grid(True, ls='--', zorder=-15); ax.legend(fontsize=24,title='In Order of Proximity');
    
    ax.set_xlim(250,3000)
    
    #ax.set_yscale('log')
    
    fig.savefig('3_Temperature_Distributions/Temperature Distribution at {}.png'.format(title_loc))
    plt.close(fig)
    
    

In [ ]:
for each in samples:
    distribution(each)

Apollo 11


  0%|                                                    | 0/19 [00:00<?, ?it/s]